In [ ]:
###################################################################################################
#
# Perform logistic regression model analysis using sklearn and keras (tensorflow) frameworks
# and perform model analysis 
# 
# Note: This notebook define the model and class weights to help the model learn from 
#       the imbalanced data
#
###################################################################################################

In [ ]:
#######################################################################################
#
# Set parameters for LR
#

BATCH_SIZE = 2048
EPOCHS = 400 
LEARNING_RATE = 0.01  # SGD default value: 0.01
MOMENTUM = 0.01   # SGD default value: 0.01

FILENAME = "../visualization/data/hospitalization_cleaned.csv"
TARGET = "../visualization/data/"

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical 
from keras.callbacks import TensorBoard
from keras.callbacks import CSVLogger
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow.keras.layers.experimental.preprocessing import IntegerLookup

import time
from time import time
import os
import tempfile

In [ ]:
# Color for graph

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
#######################################################################################
#
# Data preprocessing
#

In [ ]:
# Read dataset

file = FILENAME
df = pd.read_csv(file)

In [ ]:
# Specify label and predictors

label = ['patient_type']
predictor = [
    'age',  # age must be the first column for data pre-processing
    'sex',
    'pneumonia',
    'diabetes',
    'copd',
    'asthma',
    'inmsupr',
    'hypertension',
    'other_disease',
    'cardiovascular',
    'obesity',
    'renal_chronic',
    'tobacco'
]

label, predictor

In [ ]:
# Review the dataset

print("Dataframe shape: {}".format(df[predictor].shape))
df[predictor][0:5] # only 5 rows. You can also use either df[predictor].head() or df[predictor].tail()

In [ ]:
# Review imbalanced class column (label)

neg, pos = np.bincount(df['patient_type'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))

In [ ]:
# Get numpy n-dimentional array (tensor) from the dataset (pandas' dataframe object)

x = df[predictor].values
y = df[label].values

# Create train, test and validation datsets (in numpy's ndim-array format) 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15)

print("train shape: [features={}, label={}] \ntest shape: [features={}, label={}] \nvalidation shape: [features={}, label={}]".format(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape))

In [ ]:
# Convert dataset using standardscaler (mean: 0, std: 1) for numberic columns

from sklearn.preprocessing import StandardScaler

In [ ]:
# Preprocess train_data

scaler_age = StandardScaler().fit(x_train[0:, 0:1])

x_train_age = scaler_age.transform(x_train[0:, 0:1])
x_train_remaining = x_train[0:, 1:]

x_train_encoded = np.concatenate((x_train_age, x_train_remaining), axis=1)

print("age column mean: {}, std: {}".format(scaler_age.mean_, scaler_age.scale_))
print("x_train encoded shape: {}".format(x_train_encoded.shape))

In [ ]:
# Preprocess test_data

x_test_age = scaler_age.transform(x_test[0:, 0:1])
x_test_remaining = x_test[0:, 1:]

x_test_encoded = np.concatenate((x_test_age, x_test_remaining), axis=1)
print("x_test encoded shape: {}".format(x_test_encoded.shape))

In [ ]:
# Preprocess val_data

x_val_age = scaler_age.transform(x_val[0:, 0:1])
x_val_remaining = x_val[0:, 1:]

x_val_encoded = np.concatenate((x_val_age, x_val_remaining), axis=1)
print("x_val encoded shape: {}".format(x_val_encoded.shape))

In [ ]:
#######################################################################################
#
# Build ml models
#

In [ ]:
# Define metrics for ml models

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

In [ ]:
# Define various ml models e.g., standard neural network and logistic regression

def build_snn_w_adam(input_dim, learning_rate = 1e-3, beta_1 = 0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, 
                     metrics=METRICS, output_bias=None):
    # initialize output bias if specified
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    model = keras.Sequential()
    model.add(layers.Dense(32, activation='relu', input_dim=input_dim))
    model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(16, activation='relu'))
#     model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid', bias_initializer=output_bias))
    
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, 
                                                beta_1=beta_1,
                                                beta_2=beta_2,
                                                epsilon=epsilon,
                                                amsgrad=amsgrad), #'adam',
                    loss=keras.losses.BinaryCrossentropy(from_logits=False),
                    metrics=metrics)

    return model

def build_snn_w_sgd(input_dim, learning_rate = 0.01, momentum=0.01, nesterov=False, metrics=METRICS, output_bias=None):
    # initialize output bias if specified
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    model = keras.Sequential()
    model.add(layers.Dense(32, activation='relu', input_dim=input_dim))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid', bias_initializer=output_bias))

    model.compile(optimizer=keras.optimizers.SGD(
                                        learning_rate=learning_rate, 
                                        momentum=momentum, 
                                        nesterov=nesterov, 
                                        name="SGD"),
                    loss=keras.losses.BinaryCrossentropy(from_logits=False),
                    metrics=metrics)
    
    return model

def build_lr_w_sgd(input_dim, learning_rate = 0.01, momentum=0.01, nesterov=False, metrics=METRICS, output_bias=None):
    # initialize output bias if specified
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    model = keras.Sequential()
    model.add(layers.Dense(1, activation='sigmoid', input_dim=input_dim, bias_initializer=output_bias))

    model.compile(optimizer=keras.optimizers.SGD(
                                        learning_rate=learning_rate, 
                                        momentum=momentum, 
                                        nesterov=nesterov, 
                                        name="SGD"),
                    loss=keras.losses.BinaryCrossentropy(from_logits=False),
                    metrics=metrics)
    
    return model


In [ ]:
#######################################################################################
#
# Find correct initial bias, checkpoint the initial weights and confirm whether 
# the bias fix helps or not
#

In [ ]:
# Find correct initial bias

initial_bias = np.log([pos/neg]) # pos and neg are calculated previously: 
print(initial_bias)

In [ ]:
# Build the model and review the structure
model_b = build_lr_w_sgd(input_dim=x_train_encoded.shape[1])
model_b.summary()

In [ ]:
# Predict the model with train dataset

model_b.predict(x=x_train_encoded, steps=10)

In [ ]:
# Evaluate previous model

results = model_b.evaluate(x=x_train_encoded, y=y_train, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

In [ ]:
# Build the model with initial_bias

model_i = build_lr_w_sgd(input_dim=x_train_encoded.shape[1], output_bias=initial_bias)
model_i.predict(x=x_train_encoded, steps=10)
results = model_i.evaluate(x=x_train_encoded, y=y_train, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

In [ ]:
# Save initial_weights

initial_weights = os.path.join(tempfile.mkdtemp(),'initial_weights')
model_i.save_weights(initial_weights)

In [ ]:
# Run models with and without initial bias

model_v = build_lr_w_sgd(input_dim=x_train_encoded.shape[1])
model_v.load_weights(initial_weights)
model_v.layers[-1].bias.assign([0.0])
zero_bias_history = model_v.fit(
    x=x_train_encoded, 
    y=y_train,
    validation_data=(x_val_encoded, y_val), 
    epochs=20,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    verbose=0)

model_v = build_lr_w_sgd(input_dim=x_train_encoded.shape[1])
model_v.load_weights(initial_weights)
careful_bias_history = model_v.fit(
    x=x_train_encoded, 
    y=y_train,
    validation_data=(x_val_encoded, y_val), 
    epochs=20,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    verbose=0)

In [ ]:
# Defint plot_loss

def plot_loss(history, label, n):
    # Use a log scale to show the wide range of values.
    plt.semilogy(history.epoch,  history.history['loss'],
               color=colors[n], label='Train '+label)
    plt.semilogy(history.epoch,  history.history['val_loss'],
          color=colors[n], label='Val '+label,
          linestyle="--")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend()

In [ ]:
# Confirm the bias fix with plot_loss graphs

plot_loss(zero_bias_history, "Zero Bias", 0)
plot_loss(careful_bias_history, "Careful Bias", 1)

In [ ]:
def plot_metrics(history):
    metrics =  ['loss', 'auc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.7,1])
#             plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

        plt.legend()


In [ ]:
# Define plot_confusion_matrix function

def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    print('No Hospitalization Correctly Detected (True Negatives): ', cm[0][0])
    print('Hospitalization Incorrectly Detected (False Positives): ', cm[0][1])
    print('No Hospitalization Missed (False Negatives): ', cm[1][0])
    print('Hospitalization Detected (True Positives): ', cm[1][1])
    print('Total Hospitalization: ', np.sum(cm[1]))

In [ ]:
# Define plot_receiver_operating_characteristic function

def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
#     plt.xlim([-0.5,20])
#     plt.ylim([80,100.5])
    plt.xlim([-0.5,100.5])
    plt.ylim([40,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [ ]:
#######################################################################################
#
# Start training by specifying class (e.g., label, y) weights
# 
# Note: we are trying to make the model to pay more attention to under-represented data
#

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.

weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
# csv logger
csv_logger = CSVLogger(TARGET + "hospitalization_logistic_regression_8_train_history.csv", append=False, separator=',')

In [ ]:
# Run the model
model_weighted = build_lr_w_sgd(learning_rate=LEARNING_RATE, momentum=MOMENTUM, nesterov=True, input_dim=x_train_encoded.shape[1])
model_weighted.load_weights(initial_weights)

weighted_history = model_weighted.fit(
    x=x_train_encoded, 
    y=y_train,
    validation_data=(x_val_encoded, y_val), 
    class_weight=class_weight, # class weight added
    epochs=EPOCHS,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    verbose=1,
    callbacks=[csv_logger])

In [ ]:
plot_metrics(weighted_history)

In [ ]:
#######################################################################################
#
# Evaluate metrics with model_weighted
# 

In [ ]:
train_predictions_weighted = model_weighted.predict(x=x_train_encoded, batch_size=BATCH_SIZE)
test_predictions_weighted = model_weighted.predict(x=x_test_encoded, batch_size=BATCH_SIZE)

In [ ]:
# Evaluate the model on the test dataset

weighted_results = model_weighted.evaluate(x=x_test_encoded, y=y_test,
                                          batch_size=BATCH_SIZE, verbose=0)

for name, value in zip(model_weighted.metrics_names, weighted_results):
    print(name, ': ', value)
print()

plot_cm(y_test, test_predictions_weighted)

#
# [RESULTS]
# 

In [ ]:
# Plot the roc

plot_roc("Train Weighted", y_train, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", y_test, test_predictions_weighted, color=colors[1], linestyle='--')

plt.legend(loc='lower right')

In [ ]:
# weights and bias
#

print("====== weights and bias ==========")
w = np.asarray(model_weighted.get_weights()[0])
b = np.asarray(model_weighted.get_weights()[1]) 
print(w, b)
print("====== e^weights odds ratio =========")
print(np.exp(w).flatten())
print("====== e^bias odds ratio =========")
print(np.exp(b).flatten())

# save it
coeff_dic = {}
for name, value in zip(predictor, w.flatten()):
    coeff_dic[name] = [value]

coeff_dic["bias"] = b.flatten()

coeff_df = pd.DataFrame(coeff_dic, columns=coeff_dic.keys())
coeff_df.to_csv(TARGET + "lr_hospital_coefficients.csv", index=False)

In [ ]:
import pickle

model_json = model_weighted.to_json()
with open(TARGET + "lr_hospital.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model_weighted.save_weights(TARGET + "lr_hospital.h5")

# save StandardScaler
pickle.dump(scaler_age, open(TARGET + "lr_hospital_age_scaler.pkl", "wb"))


In [ ]:
# generate test results in csv file

metrics_dic = {}
for name, value in zip(model_weighted.metrics_names, weighted_results):
    metrics_dic[name] = [value]

metrics_df = pd.DataFrame(metrics_dic, columns=model_weighted.metrics_names)
metrics_df.to_csv(TARGET + "hospitalization_logistic_regression_8_test_results.csv", index=False)

In [ ]:
# generate train and test rocs in csv file

fp, tp, threshold = sklearn.metrics.roc_curve(y_train, train_predictions_weighted)
roc_dic = {
    "threshold": threshold,
    "fp": fp,
    "tp": tp
}
roc_df = pd.DataFrame(roc_dic, columns=["threshold", "fp", "tp"])
roc_df.sort_values(["threshold"]).to_csv(TARGET + "hospitalization_logistic_regression_8_train_roc.csv", index=False)

fp, tp, threshold = sklearn.metrics.roc_curve(y_test, test_predictions_weighted)
roc_dic = {
    "threshold": threshold,
    "fp": fp,
    "tp": tp
}
roc_df = pd.DataFrame(roc_dic, columns=["threshold", "fp", "tp"])
roc_df.sort_values(["threshold"]).to_csv(TARGET + "hospitalization_logistic_regression_8_test_roc.csv", index=False)

In [ ]:
#######################################################################################